## Window Functions

Window functions perform calculations across a set of rows related to the current row, without collapsing the result set like aggregate functions do. They're called "window" functions because they operate over a "window" of rows.

Window functions allow you to maintain the values of your original table while displaying grouped or summative information alongside in another column. This is why many Data Scientists and Data Engineers love to use window functions for complex data analysis.

### Basic Syntax
```sql
function_name([arguments]) OVER (
    [PARTITION BY partition_expression]
    [ORDER BY sort_expression [ASC|DESC]]
    [ROWS or RANGE frame_clause]
)
```

### Key Tips
- `PARTITION BY`: Divides result set into partitions (like GROUP BY but doesn't collapse rows)
- `ORDER BY`: Defines the order for the window function (required for ranking and some aggregates)
- Frame Clause: Defines which rows are included in calculations (important for running totals)
- Window functions execute after `WHERE`, `GROUP BY`, and `HAVING` clauses
- Window functions execute before `ORDER BY` and `LIMIT` clauses
- You cannot use window functions in `WHERE` clauses (use a CTE or subquery instead)
- Multiple window functions can reference the same OVER clause using the `WINDOW` keyword

### Ranking Functions
#### `ROW_NUMBER()`
Assigns a unique sequential integer to rows within a position.
```sql
-- Assign a unique number to each employee within their department
SELECT 
    employee_id,
    department,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as row_num
FROM employees;
-- Result: Each employee gets 1, 2, 3... within their department
```
#### `RANK()`
Assigns a rank with gaps when there are ties.
```sql
-- Rank employees by salary, with gaps for ties
SELECT 
    employee_id,
    salary,
    RANK() OVER (ORDER BY salary DESC) as rank
FROM employees;
-- If two employees tie for 2nd place, next rank is 4 (not 3)
```
#### `DENSE_RANK()`
Assigns a rank without gaps when there are ties.
```sql
-- Rank employees by salary, without gaps for ties
SELECT 
    employee_id,
    salary,
    DENSE_RANK() OVER (ORDER BY salary DESC) as dense_rank
FROM employees;
-- If two employees tie for 2nd place, next rank is 3
```
#### `NTILE(n)`
Distributes rows into a specified number of groups.
```sql
-- Divide employees into 4 quartiles based on salary
SELECT 
    employee_id,
    salary,
    NTILE(4) OVER (ORDER BY salary) as salary_quartile
FROM employees;
-- Returns 1, 2, 3, or 4 to indicate which quartile
```

### Aggregate Window Functions
#### `SUM()`, `AVG()`, `COUNT()`, `MIN()`, `MAX()`
```sql
-- Calculate running total of sales
SELECT 
    order_date,
    amount,
    SUM(amount) OVER (ORDER BY order_date) as running_total
FROM orders;

-- Compare individual salary to department average
SELECT 
    employee_id,
    department,
    salary,
    AVG(salary) OVER (PARTITION BY department) as dept_avg_salary,
    salary - AVG(salary) OVER (PARTITION BY department) as diff_from_avg
FROM employees;

-- Count of orders per customer with order details preserved
SELECT 
    customer_id,
    order_id,
    order_date,
    COUNT(*) OVER (PARTITION BY customer_id) as total_customer_orders
FROM orders;
```

### Value Functions
#### `LAG()`
Accesses data from a previous row.
```sql
-- Compare current month sales to previous month
SELECT 
    month,
    sales,
    LAG(sales, 1) OVER (ORDER BY month) as previous_month_sales,
    sales - LAG(sales, 1) OVER (ORDER BY month) as month_over_month_change
FROM monthly_sales;
-- LAG(column, offset, default_value)
```
#### `LEAD()`
Accesses data from a following row.
```sql
-- Compare current price to next day's price
SELECT 
    date,
    stock_price,
    LEAD(stock_price, 1) OVER (ORDER BY date) as next_day_price,
    LEAD(stock_price, 1) OVER (ORDER BY date) - stock_price as price_change
FROM stock_prices;
```
#### `FIRST_VALUE()`
Returns the first value in the window frame.
```sql
-- Compare each employee's salary to the highest in their department
SELECT 
    employee_id,
    department,
    salary,
    FIRST_VALUE(salary) OVER (
        PARTITION BY department 
        ORDER BY salary DESC
    ) as highest_dept_salary
FROM employees;
```
#### `LAST_VALUE()`
Returns the last value in the window frame.
```sql
-- Get the most recent order date per customer (requires frame specification)
SELECT 
    customer_id,
    order_date,
    LAST_VALUE(order_date) OVER (
        PARTITION BY customer_id 
        ORDER BY order_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        -- `RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` specifies the frame for our window function as the current partition and thus returns the highest number of posts in one month for each user.
    ) as most_recent_order
FROM orders;
-- Note: Frame clause is important with LAST_VALUE
```
#### `NTH_VALUE()`
Returns the nth value in the window frame.
```sql
-- Get the second highest salary in each department
SELECT DISTINCT
    department,
    NTH_VALUE(salary, 2) OVER (
        PARTITION BY department 
        ORDER BY salary DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as second_highest_salary
FROM employees;
```

### Math and Date Functions
#### Math operators
- `+` addition
- `-` subtraction
- `/` division
- `*` multiplication
- `%` modulo (returns the remainder)
#### Math functions:
- `ABS()`: returns the absolute value of the input expression
- `CAST()`: converts an expression into another data type
    - `CAST(expr AS type-name)`
    - CAST() can also be used to convert TEXT values into a numerical value like REAL or INTEGER. When doing so, it will only consider any valid prefix at the beginning of the string that represents a numerical value.
        - CAST('3.14 is pi' AS REAL) will convert the TEXT value '3.14 is pi' into type REAL, considering only the prefix '3.14' and ignoring ' is pi', resulting in 3.14
#### Date and time functions:
- `DATETIME()`: returns the date and time of a time string
    - `DATETIME('now', 'localtime')` to obtain the date and time converted to your local timezone
- `DATE()`: returns the date portion of a time string
- `TIME()`: returns the time portion of a time string
- `STRFTIME()`: returns a formatted date
##### Modifiers
Syntax: `DATETIME(timestring, modifier1, modifier2, ...)`
- `'start of year'`: shifts the date to the beginning of the current year.
- `'start of month'`: shifts the date to the beginning of the current month.
- `'start of day'`: shifts the date to the beginning of the current day.
- `'+-N years'`: offsets the year
- `'+-N months'`: offsets the month
- `'+-N days'`: offsets the day
- `'+-N hours'`: offsets the hour
- `'+-N minutes'`: offsets the minute
- `'+-N seconds'`: offsets the second

Syntax: `STRFTIME(format, timestring, modifier1, modifier2, ...)`
- `%Y` returns the year (YYYY)
- `%m` returns the month (01-12)
- `%d` returns the day of month (01-31)
- `%H` returns the hour (00-23)
- `%M` returns the minute (00-59)
- `%S` returns the second (00-59)

### Window Frame Clauses
Define which rows are included in the window for each calculation.

#### `ROWS` vs `RANGE`
```sql
-- ROWS: Physical number of rows
SELECT 
    order_date,
    amount,
    SUM(amount) OVER (
        ORDER BY order_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as three_day_moving_sum
FROM orders;
-- Includes current row and 2 physical rows before it

-- RANGE: Logical range based on values
SELECT 
    order_date,
    amount,
    SUM(amount) OVER (
        ORDER BY order_date
        RANGE BETWEEN INTERVAL '7' DAY PRECEDING AND CURRENT ROW
    ) as seven_day_total
FROM orders;
-- Includes all rows within 7 days before current row's date
```
#### Common Frame Specifications
```sql
-- Default frame (when ORDER BY is specified)
-- RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW

-- All rows in partition
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING

-- Current row only
ROWS BETWEEN CURRENT ROW AND CURRENT ROW

-- Moving average of last 3 rows
ROWS BETWEEN 2 PRECEDING AND CURRENT ROW

-- Centered moving average
ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING

-- All following rows
ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
```

### Practical Examples
#### Running Calculations
```sql
-- Running total and running average
SELECT 
    order_id,
    order_date,
    amount,
    SUM(amount) OVER (ORDER BY order_date) as running_total,
    AVG(amount) OVER (ORDER BY order_date) as running_avg,
    COUNT(*) OVER (ORDER BY order_date) as order_count
FROM orders;
```
#### Top N Per Group
```sql
-- Get top 3 earners per department
WITH ranked_employees AS (
    SELECT 
        employee_id,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as rn
    FROM employees
)
SELECT employee_id, department, salary
FROM ranked_employees
WHERE rn <= 3;
```
#### Percentage of Total
```sql
-- Calculate each product's percentage of total sales
SELECT 
    product_id,
    sales_amount,
    SUM(sales_amount) OVER () as total_sales,
    ROUND(100.0 * sales_amount / SUM(sales_amount) OVER (), 2) as pct_of_total
FROM product_sales;
```
#### Year-over-Year Comparison
```sql
-- Compare sales to same month previous year
SELECT 
    year,
    month,
    sales,
    LAG(sales, 12) OVER (ORDER BY year, month) as sales_last_year,
    sales - LAG(sales, 12) OVER (ORDER BY year, month) as yoy_change,
    ROUND(100.0 * (sales - LAG(sales, 12) OVER (ORDER BY year, month)) / 
          LAG(sales, 12) OVER (ORDER BY year, month), 2) as yoy_pct_change
FROM monthly_sales;
```
#### Removing Duplicates (Keep First)
```sql
-- Remove duplicates, keeping the first occurrence
WITH numbered_rows AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY email ORDER BY created_date) as rn
    FROM users
)
SELECT * 
FROM numbered_rows
WHERE rn = 1;
```

## User Churn
Churn rate is the percent of subscribers that have canceled within a certain period, usually a month. For a user base to grow, the churn rate must be less than the new subscriber rate for the same period. 

The churn rate is the number of these users who cancel during the month divided by the total number.

```sql
SELECT 1.0 *
(
    -- NUMERATOR
  SELECT COUNT(*)
  FROM subscriptions
  WHERE subscription_start < '2017-01-01'
  AND (
    subscription_end
    BETWEEN '2017-01-01'
    AND '2017-01-31'
  )
) / 
(
    -- DENOMINATOR
  SELECT COUNT(*)
  FROM subscriptions
  WHERE subscription_start < '2017-01-01'
  AND (
    (subscription_end >= '2017-01-01')
    OR (subscription_end IS NULL)
  )
)
AS result;
```